In [1]:
from captcha_utils.generator import *
from models.cnn_ctc import *


train_path = 'E:\\traindata\\captcha_origin\\train'
validate_path = 'E:\\traindata\\captcha_origin\\test'
img_w = 200
img_h = 60
channel = 1
downsample_factor = 4


model, test_func = icp_model()
image_gen = TextImageGenerator(train_path, validate_path, img_w, img_h, channel, downsample_factor)
# 测试
# test, _ = next(image_gen.get_next_train(10))
# x_input = test['the_input']
# print(x_input.shape)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


E:\Github\machine_learning\image\qrcode\models\cnn_ctc.py:64: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("th..., outputs=Tensor("so...)`
  base_model = Model(input=input_data, output=y_pred)


In [2]:
import keras.callbacks
import datetime

class VizCallback(keras.callbacks.Callback):

    def __init__(self, run_name, test_func, text_img_gen, output_dir, num_display_words=6):
        self.test_func = test_func
        self.output_dir = os.path.join(
            output_dir, run_name)
        self.text_img_gen = text_img_gen
        self.num_display_words = num_display_words
        if not os.path.exists(self.output_dir):
            os.makedirs(self.output_dir)

    def on_epoch_end(self, epoch, logs={}):
        if epoch % 10 == 0:  # 每10个周期计算一次正确率
            word_batch = next(self.text_img_gen)[0]
            res = decode_batch(self.test_func, TextImageGenerator.labels_to_text, word_batch['the_input'])
            acc = 0
            total = word_batch['the_input'].shape[0]
            for i in range(total):
                if word_batch['source_str'][i].lower() == res[i].lower():
                    acc += 1
            acc_ratio = 100 * acc / total
            print('正确率: %0.5f' % acc_ratio)
            if acc_ratio > 50:
                self.model.save_weights(os.path.join(self.output_dir, 'weights%02d_acc_%0.5f.h5' % (epoch, acc_ratio)))
        word_batch = next(self.text_img_gen)[0]
        res = decode_batch(self.test_func, TextImageGenerator.labels_to_text, word_batch['the_input'][0:self.num_display_words])
        if word_batch['the_input'][0].shape[0] < 256:
            cols = 2
        else:
            cols = 1
        for i in range(self.num_display_words):
            plt.subplot(self.num_display_words // cols, cols, i + 1)
            if K.image_data_format() == 'channels_first':
                the_input = word_batch['the_input'][i, 0, :, :]
            else:
                the_input = word_batch['the_input'][i, :, :, 0]
            plt.imshow(the_input, cmap='Greys_r')
            plt.xlabel('T = \'%s\' Decoed = \'%s\'' % (word_batch['source_str'][i], res[i]))
        plt.savefig(os.path.join(self.output_dir, 'e%02d.png' % (epoch)))
        plt.close()

        
        
vic = VizCallback(datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S'), test_func, image_gen.get_next_val(), 
                  output_dir='E:\\Github\\machine_learning\\image\\qrcode\\logs')

In [3]:
callbacks = [vic]
train(model, image_gen.get_next_train(100), image_gen.get_next_val(200), callbacks=callbacks, epochs=400)

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/400
100/100 [==============================] - 31s 306ms/step - loss: 27.4623 - val_loss: 21.6013
正确率: 0.00000
Epoch 2/400
100/100 [==============================] - 25s 252ms/step - loss: 21.1757 - val_loss: 21.0010
Epoch 3/400
100/100 [==============================] - 25s 251ms/step - loss: 20.0390 - val_loss: 19.2213
Epoch 4/400
100/100 [==============================] - 25s 254ms/step - loss: 18.4334 - val_loss: 17.8782
Epoch 5/400
100/100 [==============================] - 26s 258ms/step - loss: 16.8794 - val_loss: 16.3157
Epoch 6/400
100/100 [==============================] - 26s 260ms/step - loss: 15.2771 - val_loss: 14.8045
Epoch 7/400
100/100 [==============================] - 26s 256ms/step - loss: 13.8840 - val_loss: 13.7927
Epoch 8/400
100/100 [==============================] - 26s 256ms/step - loss: 12.6950 - val_loss: 13.2362
Epoch 9/400
100/100 [==============================] - 26s 25

100/100 [==============================] - 25s 251ms/step - loss: 0.0044 - val_loss: 18.4991
Epoch 75/400
100/100 [==============================] - 25s 249ms/step - loss: 0.0043 - val_loss: 18.4982
Epoch 76/400
100/100 [==============================] - 26s 256ms/step - loss: 0.0042 - val_loss: 18.7171
Epoch 77/400
100/100 [==============================] - 25s 253ms/step - loss: 0.0042 - val_loss: 18.6991
Epoch 78/400
100/100 [==============================] - 26s 257ms/step - loss: 0.0041 - val_loss: 18.5486
Epoch 79/400
100/100 [==============================] - 26s 257ms/step - loss: 0.0040 - val_loss: 18.7808
Epoch 80/400
100/100 [==============================] - 25s 252ms/step - loss: 0.0040 - val_loss: 18.9002
Epoch 81/400
100/100 [==============================] - 25s 253ms/step - loss: 0.0039 - val_loss: 18.8137
正确率: 1.00000
Epoch 82/400
100/100 [==============================] - 25s 253ms/step - loss: 0.0038 - val_loss: 18.7298
Epoch 83/400
100/100 [========================

100/100 [==============================] - 25s 253ms/step - loss: 0.0019 - val_loss: 19.8174
Epoch 151/400
100/100 [==============================] - 25s 252ms/step - loss: 0.0019 - val_loss: 19.6841
正确率: 4.00000
Epoch 152/400
100/100 [==============================] - 25s 251ms/step - loss: 0.0018 - val_loss: 19.8699
Epoch 153/400
100/100 [==============================] - 25s 251ms/step - loss: 0.0018 - val_loss: 19.4806
Epoch 154/400
100/100 [==============================] - 25s 252ms/step - loss: 0.0018 - val_loss: 20.0235
Epoch 155/400
100/100 [==============================] - 25s 251ms/step - loss: 0.0018 - val_loss: 19.5587
Epoch 156/400
100/100 [==============================] - 25s 252ms/step - loss: 0.0018 - val_loss: 19.6746
Epoch 157/400
100/100 [==============================] - 25s 250ms/step - loss: 0.0018 - val_loss: 19.9631
Epoch 158/400
100/100 [==============================] - 25s 252ms/step - loss: 0.0018 - val_loss: 19.8170
Epoch 159/400
100/100 [===============

100/100 [==============================] - 26s 258ms/step - loss: 0.0012 - val_loss: 20.2318
Epoch 226/400
100/100 [==============================] - 25s 252ms/step - loss: 0.0012 - val_loss: 19.8852
Epoch 227/400
100/100 [==============================] - 26s 255ms/step - loss: 0.0012 - val_loss: 20.2358
Epoch 228/400
100/100 [==============================] - 26s 255ms/step - loss: 0.0012 - val_loss: 20.4593
Epoch 229/400
100/100 [==============================] - 25s 255ms/step - loss: 0.0012 - val_loss: 20.2946
Epoch 230/400
100/100 [==============================] - 27s 268ms/step - loss: 0.0012 - val_loss: 20.2464
Epoch 231/400
100/100 [==============================] - 26s 256ms/step - loss: 0.0012 - val_loss: 20.4081
正确率: 2.00000
Epoch 232/400
 31/100 [========>.....................] - ETA: 16s - loss: 0.0012

KeyboardInterrupt: 